### Work division
#### The work for this notebook is done and compiled by Nemish Murawat.

#### Libararies Required

In [1]:
# Import Requirements
import pandas as pd
from tqdm import tqdm as tq
import numpy as np
import matplotlib.pyplot as plt
from pre_processing_posts import pre_processing_posts as pre_posts
import datetime
from collections import Counter

### RQ7.1 What's the probability that a post receives more than 20% "likes" of the number of followers a user has?

### *Approach*:
#### For this part I would be working with two dataset *Instagram_Profiles.csv* to get followers information and *Instagram_Posts.csv* dataset to get user's likes information.
#### Step-1
>##### I read the complete dataset of the Instagram_Profiles dataset with columns [sid, followers] as the dataset size is small. 
>##### I drop rows with NaN values

In [2]:
#Drop NaN valued rows
df_profiles=pd.read_csv('/home/ec2-user/SageMaker/Data/instagram_profiles.csv',usecols=['sid','followers'],sep='\t')
df_profiles=df_profiles.dropna()

#### Step-2 
>##### I create a function that takes in as input number of likes and number of followers for a user and returns 1 or 0 to highlight those users which follow the constraint.


In [3]:

def compare_likes_followers(x,y):
'''Arguments':#This function takes as input value of number of like(x) and Number of followers(y) for a user

'Returns': '1 if the number of likes is greater than 20% of the user else 0 the other way round'''

    if x>(0.2)*y:
        return 1
    else:
        return 0

#### Step-3
> ##### I iterate over the Instagram_Posts dataset over small-size chunks and just read the 'sid_profile' and 'numbr_likes' columns.
> ##### I drop those rowns in chunks which has Nan Values.
> ##### I merge df_profile and chunk dataframes on columns ['sid_profiles'] and ['sid'] to get those users which are present in both the datasets.( Removing inconsistencies)
>> ##### Given on Kaggle Web page of the datasets that 'sid_profile' is made from the 'sid' of the profiles. Hence used this columns to get a match.
> ##### I apply the compare_like_followers function on the merged dataframe on columns['followers' and 'likes'] to get a dataframe which helps me the count of the users which follow the given constraint.
>##### Finally, in the next cell I output the recieved result for the question.

In [ ]:

for i,chunk in tq(enumerate(pd.read_csv('/home/ec2-user/SageMaker/Data/instagram_posts.csv',sep='\t',usecols=['sid_profile','numbr_likes'],chunksize=100000,keep_default_na=True,na_values=' '))):
        chunk=chunk.dropna(how='any')
        chunk=chunk.rename(columns={'sid_profile':'sid'})
        df_iter=pd.merge(chunk,df_profiles,on='sid')
        try:
            x=df_iter.apply( lambda row: compare_likes_followers(int(row['numbr_likes']),int(row['followers'])),axis=1)
        except:
            print('it still does not drop '' ')
        sum1=sum1+sum(x) if i!=0 else sum(x)
        size=size+len(x) if i!=0 else len(x) 
        
        
   
        
        

In [7]:
print('The probability that a post receives more than 20% "likes" of the number of followers a user has is '+str(round(sum1*100/size,2))+'%')    

The probability that a post receives more than 20% "likes" of the number of followers a user has is 15.24%


### RQ7.2 Do users usually return to locations? Extract the probability that a user returns to a site after having posted it in the past. Does that probability make sense to you? Explain why or why not.

##### Starting idea is to to get use Locations. csv and Posts.csv simultaneously. Since Locations dataset is small in size, I read the complete Location dataset in one-go. Whereas for the Posts dataset, I read the dataset in multiple chunks.

### *Approach*:
> ##### Step-1. While reading the dataset in chunk for Posts I extract the profile id and location_id columns whereas just the 'id' column for the location dataset.
> ##### Step-2. Now, for each chunk I try to match the Posts['location_id'] with Location['id'] so that I can query location information from the matched location['id'].
> ##### Step-3. Now each user would have a unique 'Profile Id' and through which I can see how many different locations the user has visited and see if the user went back to the sampe place again.
> ##### Step-4. Now the final array containing profile_id and location id is quite large and to improve the efficiency.I adopt a following procedure:
>> #####        1.I create a string variable using each 'profile_id'-'location_id' (combined) to define a new new variable.       
>> #####        2.Now,I pass this variable to a counter object to get the frequencies of the formed string.        
>> #####        3.Those string which have counts>1 means that the a particular user have revisited the place.        
>> #####        4.Hence I find the sum of all the strings which have more than count=1 and sum of all the counts to estimate the probability.
        
        
#### Assumption:
 
> ##### Every row in *Instagram_location.csv* is distinct.
        



In [2]:
#Drop NaN valued rows #Profiles(Step-1)

df_locations=pd.read_csv('/home/ec2-user/SageMaker/Data/instagram_locations.csv',usecols=['id'],sep='\t')
df_locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1022658 entries, 0 to 1022657
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   id      1022658 non-null  int64
dtypes: int64(1)
memory usage: 7.8 MB


In [4]:
#Step-1 and Step-2
for i,chunk in tq(enumerate(pd.read_csv('/home/ec2-user/SageMaker/Data/instagram_posts.csv',sep='\t',usecols=['profile_id','location_id'],chunksize=1000000,keep_default_na=True))):
           df_final=pd.merge(chunk,df_locations,left_on='location_id',right_on='id')
           df_final=df_final.drop(['id'],axis=1)
           x=np.concatenate([x,df_final.values],axis=0) if i!=0 else df_final.values

43it [06:59,  9.76s/it]


In [5]:
#Step-3 and Step-4
profile_id=x.copy()[:,0]
location_id=x.copy()[:,1]
z=[]
for i,j in tq(enumerate(profile_id)):
    z.append(str(j)+'-'+str(location_id[i]))

c=Counter(z)
count_freq=0
for i,j in tq(enumerate(c.values())):
    if j>1:
        count_freq += j
print('The probability that a user returns to a site after having posted it in the past is {}%'.format((count_freq/sum(c.values()))*100))    

28063765it [00:53, 525698.56it/s]
20074052it [00:05, 3566907.80it/s]

The probability that a user returns to a site after having posted it in the past is 38.759571283468205%
